## Search, Retrieve & Summarize using Text Bison

#### Imports 

In [1]:

from langchain.chains import MapReduceDocumentsChain
from langchain.embeddings import VertexAIEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.chains import ReduceDocumentsChain
from langchain.chains import StuffDocumentsChain
from langchain.embeddings.base import Embeddings
from langchain.llms.vertexai import VertexAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.vectorstores import FAISS
from typing import List
from tqdm import tqdm
import logging
import os 

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Setup

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './credentials/vai-key.json'

LOCAL_INPUT_DIR = './DATA/INPUT'
LOCAL_OUTPUT_DIR = './DATA/OUTPUT'

TEXT_GEN_MODEL_NAME = 'text-bison@latest'
TEXT_EMBED_MODEL_NAME = 'textembedding-gecko'

FILE_NAME = 'file-2'

In [4]:
class MyVertexAIEmbeddings(VertexAIEmbeddings, Embeddings):
    model_name = 'textembedding-gecko'
    max_batch_size = 5
    
    def embed_segments(self, segments: List) -> List:
        embeddings = []
        for i in tqdm(range(0, len(segments), self.max_batch_size)):
            batch = segments[i: i+self.max_batch_size]
            embeddings.extend(self.client.get_embeddings(batch))
        return [embedding.values for embedding in embeddings]
    
    def embed_query(self, query: str) -> List:
        embeddings = self.client.get_embeddings([query])
        return embeddings[0].values

In [5]:
text_embeder = MyVertexAIEmbeddings(model_name=TEXT_EMBED_MODEL_NAME)

#### Encode and index text content for pages 

In [6]:
def extract_metadata(record: dict, metadata: dict) -> dict:
    metadata['doc_name'] = record.get('doc_name')
    metadata['page_num'] = record.get('page_num')
    return metadata


loader = JSONLoader(file_path=f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/TEXT/{FILE_NAME}.jsonl', 
                    jq_schema='.', 
                    metadata_func=extract_metadata, 
                    content_key='page_content', 
                    json_lines=True)

segments = loader.load()  # here segments correspond to pages 

In [7]:
vector_store = FAISS.from_documents(segments, text_embeder)

#### Setup Retrieval Chain

In [8]:
retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k': 10})
retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x12c1e7e80>, search_type='mmr', search_kwargs={'k': 10})

In [9]:
llm = VertexAI(max_output_tokens=2048, 
               model_name=TEXT_GEN_MODEL_NAME,
               temperature=0,
               top_p=0.8,
               top_k=40,
               verbose=True)
llm

VertexAI(cache=None, verbose=True, callbacks=None, callback_manager=None, tags=None, metadata=None, project=None, location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='text-bison@latest', client=<vertexai.language_models._language_models._PreviewTextGenerationModel object at 0x12a4b95b0>, temperature=0.0, max_output_tokens=2048, top_p=0.8, top_k=40, credentials=None, tuned_model_name=None)

#### Retrieve matching pages related to `derivative contracts`

In [10]:
query = "derivative contracts"

In [11]:
pages = retriever.get_relevant_documents(query)

for page in pages:
    logger.info(f'Content => {page.page_content}')
    logger.info(f'Metadata => {page.metadata}')
    logger.info(f"\n{'-' * 100}\n")

Content =>   
  
 (ii) The maturity factor of a derivative cont ract that is not subject to a variation margin 
agreement, or derivative contracts under which the counterparty is not required to post variation 
margin, is determined by the following formula: 
min {ܯ250 }Maturity factor = ඨ 
250 
Where M equals the greater of 10 business days and the remaining maturity of the 
contract, as measured in business days. 
(iii) For purposes of paragraph (i)(4) of this section, if a [BANKING ORGANIZATION] 
has elected pursuant to paragraph (e)(5) of this s ection to treat a deriva tive contract that is a 
cleared transaction that is not s ubject to a variation margin agreement as one that is subject to a 
variation margin agreement, the [BANKING OR GANIZATION] must treat the derivative 
contract as subject to a variation margin agreement with maturity factor as determined according 
to paragraph (i)(4)(i) of this section, and daily s ettlement does not change the end date of the 
period refer

#### Summarize the matched pages into a detailed summary using `MapReduce`

In [12]:
# Map chain
map_template = """You are a Derivatives Risk Analyst. Your job is to create a detailed SUMMARY of financial documents.
Your summary should include all important facts, math formulas, and technical details, and leave out extra information.
Your goal is to provide a clear and precise SUMMARY of the PAGE below that shows your knowledge of finance, with high accuracy and quality.
---
PAGE => {PAGE}
---
SUMMARY =>"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, 
                     prompt=map_prompt, 
                     verbose=True)

# Reduce chain
reduce_template = """The following is set of SUMMARIES:
---
SUMMARIES => {SUMMARIES}
---
Combine the summaries into a single cohesive CONSOLIDATED summary without distilling or removing any information. Add sections wherever applicable to clearly delineate topics or themes.
CONSOLIDATED SUMMARY =>"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, 
                        prompt=reduce_prompt, 
                        verbose=True)

combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, 
                                              document_variable_name='SUMMARIES', 
                                              verbose=True)

reduce_documents_chain = ReduceDocumentsChain(combine_documents_chain=combine_documents_chain, 
                                              collapse_documents_chain=combine_documents_chain, 
                                              token_max=2048, 
                                              verbose=True)

map_reduce_chain = MapReduceDocumentsChain(llm_chain=map_chain, 
                                           reduce_documents_chain=reduce_documents_chain, 
                                           document_variable_name='PAGE', 
                                           return_intermediate_steps=False, 
                                           verbose=True)

In [13]:
completion = map_reduce_chain.run(pages)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a Derivatives Risk Analyst. Your job is to create a detailed SUMMARY of financial documents.
Your summary should include all important facts, math formulas, and technical details, and leave out extra information.
Your goal is to provide a clear and precise SUMMARY of the PAGE below that shows your knowledge of finance, with high accuracy and quality.
---
PAGE =>   
  
 (ii) The maturity factor of a derivative cont ract that is not subject to a variation margin 
agreement, or derivative contracts under which the counterparty is not required to post variation 
margin, is determined by the following formula: 
min {ܯ250 }Maturity factor = ඨ 
250 
Where M equals the greater of 10 business days and the remaining maturity of the 
contract, as measured in business days. 
(iii) For purposes of paragraph (i)(4) of this section, if a [BANKING ORGANIZATION] 
has elected pursuant to

/Users/arunpshankar/Desktop/Projects/citi-icg-pocs/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (1591 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:
The following is set of SUMMARIES:
---
SUMMARIES =>  1. Maturity factor of a derivative contract that is not subject to a variation margin agreement, or derivative contracts under which the counterparty is not required to post variation margin, is determined by the formula: 
Maturity factor = min{250/M, 1}
Where M equals the greater of 10 business days and the remaining maturity of the contract, as measured in business days.

2. For purposes of paragraph (i)(4) of this section, if a [BANKING ORGANIZATION] has elected pursuant to paragraph (e)(5) of this section to treat a derivative contract that is a cleared transaction that is not subject to a variation margin agreement as one that is subject to a variation margin agreement, the [BANKING ORGANIZATION] must treat the derivative contract as subject to a variation margin agreement with maturity factor as determined according to paragraph (i)(4)(i) of this section, and daily set

In [14]:
completion = completion.strip()
logger.info(completion)

**Maturity Factor** 

Maturity factor of a derivative contract that is not subject to a variation margin agreement, or derivative contracts under which the counterparty is not required to post variation margin, is determined by the formula: 
Maturity factor = min{250/M, 1}
Where M equals the greater of 10 business days and the remaining maturity of the contract, as measured in business days.

**Treatment of Cleared Transactions** 

For purposes of paragraph (i)(4) of this section, if a [BANKING ORGANIZATION] has elected pursuant to paragraph (e)(5) of this section to treat a derivative contract that is a cleared transaction that is not subject to a variation margin agreement as one that is subject to a variation margin agreement, the [BANKING ORGANIZATION] must treat the derivative contract as subject to a variation margin agreement with maturity factor as determined according to paragraph (i)(4)(i) of this section, and daily settlement does not change the end date of the period refere

In [15]:
SUMMARY_OUT_PATH = f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/SUMMARY'
os.makedirs(SUMMARY_OUT_PATH, exist_ok=True)

with open(f'{SUMMARY_OUT_PATH}/{FILE_NAME}-summary-vai.txt', 'w') as f:
    f.write(completion)